### StuffDocumentChain Text Summarization

##### Stuff: summarize in a single LLM call
##### StuffDocumentChain is one of several document chain types in LangChain designed for handling multiple documents or long texts. Specifically:

Use Cases:
Short to Medium-Length Texts: Ideal for summarizing documents or chapters that fit within the model's token limit but are too long for simple queries.
Research Papers: For summarizing multiple sections of a research paper that are contextually related.
Educational Material: Summarizing lecture notes or textbook chapters to create study aids.

In [37]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [38]:
import os

In [39]:
from langchain_groq import ChatGroq
api_key='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'
llm=ChatGroq(groq_api_key=api_key,model="gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x17f38ae10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x17da2fe90>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [40]:
template = """
Write a Concise and Short Summary of the Following Speech,
Speech :{text}
"""

prompt = PromptTemplate(input_variables = ['text'],
                        template=template)

In [41]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nWrite a Concise and Short Summary of the Following Speech,\nSpeech :{text}\n')

In [42]:
from langchain.chains.summarize import load_summarize_chain

In [52]:
chain = load_summarize_chain(llm,chain_type='stuff',prompt=prompt, verbose=True)
chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a Concise and Short Summary of the Following Speech,
Speech :A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online int

'In his departing speech, President A.P.J. Abdul Kalam reflects on his five years in office and outlines his vision for India\'s future. He emphasizes the importance of youth empowerment, rural development, and societal transformation through technology. \n\n**Key takeaways:**\n\n* **Youth Aspiration:** Kalam highlights the desire of India\'s youth to live in a developed nation by 2020, emphasizing their potential as a driving force for change.\n* **Empowering Villages:** He stresses the need to empower villages through connectivity, infrastructure, and economic opportunities to bridge the rural-urban divide.\n* **Technological Advancement:** Kalam champions the use of technology, particularly "Pan African e-Network," to connect people, improve healthcare and education, and foster societal progress.\n* **National Defense:** He expresses pride in India\'s armed forces and their commitment to protecting the nation.\n* **Moral Values:** Kalam emphasizes the importance of value-based educa

### Map Reduce to Summarize Large Documents

In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(docs)
text_splitter

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [45]:
len(text_splitter)

13

In [46]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nWrite a Concise and Short Summary of the Following Speech,\nSpeech :{text}\n')

In [47]:
chunk_prompt = """
Please Summarize the Below Speech:
Speech :`{text}'
"""

map_prompt_template = PromptTemplate(input_variables = ['text'],
                        template=template)

In [48]:
final_prompts = '''
Provide the final Summary of the Speech with these Important points.
Add a Motivation Title, Start the Precise Summary with an introduction and Provide the summary in number points for the Speech

Speech{text}
'''

final_prompt_template = PromptTemplate(input_variables=["text"], template=final_prompts)

In [49]:
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final Summary of the Speech with these Important points.\nAdd a Motivation Title, Start the Precise Summary with an introduction and Provide the summary in number points for the Speech\n\nSpeech{text}\n')

In [53]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt = map_prompt_template,
    combine_prompt = final_prompt_template,
    verbose=True
)

output= summary_chain.run(text_splitter)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a Concise and Short Summary of the Following Speech,
Speech :A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online

'##  Motivational Title:  A Vision for a Developed India:  Kalam\'s Farewell Speech \n\n**Introduction:**  \n\nA.P.J. Abdul Kalam\'s departing address as President of India is a poignant reflection on his five years of service and a powerful call to action for a brighter future. In this speech, he distills key lessons learned, outlines a blueprint for national development, and inspires the youth to lead the charge towards a prosperous India by 2020. \n\n**Summary:**\n\n1.  **Youth Aspiration as the Engine of Development:** Kalam highlights the importance of youth dreams and goals, illustrated by a young girl\'s question about India\'s development timeline. He emphasizes their role as agents of change.\n\n2. **Empowering Villages for National Progress:**  Kalam champions the empowerment of India\'s 600,000 villages, drawing inspiration from a successful village council in Nagaland. He envisions connecting villages through physical, electronic, and knowledge networks for economic growth.

In [54]:
output

'##  Motivational Title:  A Vision for a Developed India:  Kalam\'s Farewell Speech \n\n**Introduction:**  \n\nA.P.J. Abdul Kalam\'s departing address as President of India is a poignant reflection on his five years of service and a powerful call to action for a brighter future. In this speech, he distills key lessons learned, outlines a blueprint for national development, and inspires the youth to lead the charge towards a prosperous India by 2020. \n\n**Summary:**\n\n1.  **Youth Aspiration as the Engine of Development:** Kalam highlights the importance of youth dreams and goals, illustrated by a young girl\'s question about India\'s development timeline. He emphasizes their role as agents of change.\n\n2. **Empowering Villages for National Progress:**  Kalam champions the empowerment of India\'s 600,000 villages, drawing inspiration from a successful village council in Nagaland. He envisions connecting villages through physical, electronic, and knowledge networks for economic growth.

## Refine Chain for Summarization

In [56]:
chain = load_summarize_chain(
    llm = llm,
    chain_type="refine",
    verbose=True
)

output_summary = chain.run(text_splitter)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many

In [58]:
output_summary

'In his departing speech, A.P.J. Abdul Kalam reflects on his five years as President of India, expressing gratitude for the experiences and interactions with people from diverse backgrounds.  \n\nHe highlights ten key messages from his time in office:\n\n1. **Accelerate development:** Empowering the youth\'s aspirations.\n2. **Empower villages.**\n3. **Mobilize rural competence for competitiveness.**\n4. **Focus on agriculture: Seed to Food.**\n5. **Overcome problems through perseverance and partnership.**\n6. **Courage in facing calamities.**\n7. **Connectivity for societal transformation.**\n8. **National defense and pride.**\n9. **Youth movement for a developed India by 2020.**\n10. **A Nation that is one of the best places to live in and is proud of its leadership.**\n\nKalam underscores the importance of youth engagement in national progress, recounting a memorable anecdote where a young girl questioned India\'s path to development. He reassured her that her dream would be taken t